# **Big Data**
#BD02 Spark DataFrames

# Dataframes

Un Spark DataFrame es una colección inmutable de datos distribuidos dentro de un clúster. Los datos dentro de un DataFrame se organizan en columnas con nombre que se pueden comparar con tablas en una base de datos relacional. Los DataFrames aprovechan los avances en el proyecto tungsteno
 y Catalyst Optimizer. Estas dos mejoras aportan la
rendimiento de PySpark a la par con el de Scala o Java.

Catalyst Optimizer se encuentra en el core de Spark SQL y potencia tanto las consultas SQL ejecutadas en los datos como en los DataFrames. El proceso comienza con la consulta que se envía al motor. Primero se optimiza el plan lógico de ejecución. Basado en el plan lógico optimizado, se derivan múltiples planes físicos y se envían a través de un optimizador de costos. Luego, el plan seleccionado y más rentable se traduce (utilizando optimizaciones de generación de código implementadas como parte del proyecto de tungsteno) en un código de ejecución optimizado basado en RDD.

Aunque Python no es un lenguaje fuertemente tipado, los DataFrames en PySpark sí lo son. A diferencia de los RDD, cada elemento de una columna DataFrame tiene un tipo específico (todos se enumeran en el submódulo pyspark.sql.types) y todos los datos deben ajustarse al esquema especificado. El Catalyst Optimizer se encuentra en el core de Spark SQL y potencia tanto las consultas SQL ejecutadas contra los datos como DataFrames. El proceso comienza con la consulta que se envía al motor. Primero se optimiza el plan lógico de ejecución. Basado en el plan lógico optimizado, se derivan múltiples planes físicos y se envían a través de un optimizador de costos. Luego, el plan seleccionado y más rentable se traduce (utilizando optimizaciones de generación de código implementadas como parte del proyecto de tungsteno) en un código de ejecución optimizado basado en RDD.

Cambiar a usar DataFrames no significa que debamos
abandonar los RDD. DataFrames todavía usa RDD, pero de objetos Row(...),  En este notebook, aprenderemos cómo interactuar con  un DataFrame.

In [3]:
# Esto solo lo utilizamos para instalar las librerias
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.5.5/spark-3.5.5-bin-hadoop3.tgz
!tar xf spark-3.5.5-bin-hadoop3.tgz
!pip install -q findspark
!pip install pyarrow==0.15.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 61.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for pyarrow (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for pyarrow
Failed to build pyarrow
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (pyarrow)


In [1]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.5-bin-hadoop3"

In [4]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

## Creemos algunos ejemplos.

In [5]:
sample_data = spark.sparkContext.parallelize([
      (1, 'MacBook Pro', 2015, '15"', '16GB', '512GB SSD', 13.75, 9.48, 0.61, 4.02)
    , (2, 'MacBook', 2016, '12"', '8GB', '256GB SSD', 11.04, 7.74, 0.52, 2.03)
    , (3, 'MacBook Air', 2016, '13.3"', '8GB', '128GB SSD', 12.8, 8.94, 0.68, 2.96)
    , (4, 'iMac', 2017, '27"', '64GB', '1TB SSD', 25.6, 8.0, 20.3, 20.8)
])

In [6]:
type(sample_data)

pyspark.rdd.RDD

# Creating DataFrames

In [7]:
sample_data_df = spark.createDataFrame(
    sample_data
    , [
        'Id'
        , 'Model'
        , 'Year'
        , 'ScreenSize'
        , 'RAM'
        , 'HDD'
        , 'W'
        , 'D'
        , 'H'
        , 'Weight'
    ]
)

In [8]:
type(sample_data_df)

pyspark.sql.dataframe.DataFrame

In [9]:
sample_data_df.take(2)

[Row(Id=1, Model='MacBook Pro', Year=2015, ScreenSize='15"', RAM='16GB', HDD='512GB SSD', W=13.75, D=9.48, H=0.61, Weight=4.02),
 Row(Id=2, Model='MacBook', Year=2016, ScreenSize='12"', RAM='8GB', HDD='256GB SSD', W=11.04, D=7.74, H=0.52, Weight=2.03)]

In [10]:
sample_data.take(1)

[(1, 'MacBook Pro', 2015, '15"', '16GB', '512GB SSD', 13.75, 9.48, 0.61, 4.02)]

In [11]:
sample_data_df.take(1)

[Row(Id=1, Model='MacBook Pro', Year=2015, ScreenSize='15"', RAM='16GB', HDD='512GB SSD', W=13.75, D=9.48, H=0.61, Weight=4.02)]

In [12]:
sample_data_df.show()

+---+-----------+----+----------+----+---------+-----+----+----+------+
| Id|      Model|Year|ScreenSize| RAM|      HDD|    W|   D|   H|Weight|
+---+-----------+----+----------+----+---------+-----+----+----+------+
|  1|MacBook Pro|2015|       15"|16GB|512GB SSD|13.75|9.48|0.61|  4.02|
|  2|    MacBook|2016|       12"| 8GB|256GB SSD|11.04|7.74|0.52|  2.03|
|  3|MacBook Air|2016|     13.3"| 8GB|128GB SSD| 12.8|8.94|0.68|  2.96|
|  4|       iMac|2017|       27"|64GB|  1TB SSD| 25.6| 8.0|20.3|  20.8|
+---+-----------+----+----------+----+---------+-----+----+----+------+



In [13]:
sample_data_df.printSchema()

root
 |-- Id: long (nullable = true)
 |-- Model: string (nullable = true)
 |-- Year: long (nullable = true)
 |-- ScreenSize: string (nullable = true)
 |-- RAM: string (nullable = true)
 |-- HDD: string (nullable = true)
 |-- W: double (nullable = true)
 |-- D: double (nullable = true)
 |-- H: double (nullable = true)
 |-- Weight: double (nullable = true)



## From JSON

In [14]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [15]:
sample_data_json_df = (
    spark
    .read
    .json('/content/drive/MyDrive/DATA SCIENCE UDD 2024/MODULO 8/people.json')
)

In [16]:
sample_data_json_df.show()

+----+--------+
| age|    name|
+----+--------+
|NULL| Michael|
|  30|    Andy|
|  19|  Justin|
|  11|     Ana|
|  44|Patricia|
|  89|     Leo|
+----+--------+



In [17]:
sample_data_json_df.printSchema()

root
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)



## From CSV

In [18]:
sample_data_csv = (
    spark
    .read
    .csv(
        '/content/drive/MyDrive/DATA SCIENCE UDD 2024/MODULO 8/DataFrames_sample.csv'
        , header=True
        , inferSchema=True)
)

In [19]:
sample_data_csv.show()

+---+-----------+-----+-----------+----+---------+-----+----+----+-------+
| Id|      Model| Year| ScreenSize| RAM|      HDD|    W|   D|   H| Weight|
+---+-----------+-----+-----------+----+---------+-----+----+----+-------+
|  1|MacBook Pro| 2015|        15"|16GB|512GB SSD|13.75|9.48|0.61|   4.02|
|  2|    MacBook| 2016|        12"| 8GB|256GB SSD|11.04|7.74|0.52|   2.03|
|  3|MacBook Air| 2016|      13.3"| 8GB|128GB SSD| 12.8|8.94|0.68|   2.96|
|  4|       iMac| 2017|        27"|64GB|  1TB SSD| 25.6| 8.0|20.3|   20.8|
+---+-----------+-----+-----------+----+---------+-----+----+----+-------+



In [20]:
sample_data_csv.printSchema()

root
 |-- Id: integer (nullable = true)
 |--  Model: string (nullable = true)
 |--  Year: integer (nullable = true)
 |--  ScreenSize: string (nullable = true)
 |--  RAM: string (nullable = true)
 |--  HDD: string (nullable = true)
 |--  W: double (nullable = true)
 |--  D: double (nullable = true)
 |--  H: double (nullable = true)
 |--  Weight: double (nullable = true)



## Acceder a RDD subyacentes

Cambiar a usar DataFrames no significa que debamos
abandonar los RDD. DataFrames todavía usa RDD, pero de objetos Row (...)

In [21]:
sample_data_df.rdd.take(1)


[Row(Id=1, Model='MacBook Pro', Year=2015, ScreenSize='15"', RAM='16GB', HDD='512GB SSD', W=13.75, D=9.48, H=0.61, Weight=4.02)]

In [22]:
sample_data.take(1)

[(1, 'MacBook Pro', 2015, '15"', '16GB', '512GB SSD', 13.75, 9.48, 0.61, 4.02)]

## <font color='green'>**Ejercicio 1**</font>

Explique en detalle el código siguiente.

1. ¿Para que utilizamos las librerias importadas?

2. ¿Cambió la estructura del dataset original?

3. ¿Para que se utiliza **row.asDict()?

1. Primero, cargamos los módulos necesarios: para trabajar con los objetos Row (...), necesitamos pyspark.sql, y luego usaremos el método .round (...), por lo que necesitamos las funciones pyspark.sql.functions submódulo.

2. A continuación, extraemos .rdd de sample_data_df. Usando la transformación .map (...), primero agregamos la columna HDD_size al esquema.  Dado que estamos trabajando con RDD, queremos conservar todas las demás columnas. Por lo tanto, primero convertimos la fila (que es un objeto Row (...)) en un diccionario usando el método .asDict (), para luego poder descomprimirlo o desempaquetarlo usando **.

https://python-reference.readthedocs.io/en/latest/docs/operators/dict_unpack.html

3. Repetimos este paso dos veces más: primero, para crear la columna HDD_type, y segundo, para crear la columna Volumen.

4. A continuación, usamos el método .toDF(...) para convertir nuestro RDD nuevamente en un DataFrame.

5. Finalmente, select(...) las columnas para poder redondear (...) la columna Volumen recién creada. El método .alias (...) produce un nombre diferente para la columna resultante.



In [23]:
def add(a=0, b=0):
  return a + b

d = {'a': 2, 'b': 3}
add(**d)

5

In [24]:
import pyspark.sql as sql
import pyspark.sql.functions as f

sample_data_transformed = (
    sample_data_df
    .rdd
    .map(lambda row: sql.Row(
        **row.asDict()
        , HDD_size=row.HDD.split(' ')[0]
        )
    )
    .map(lambda row: sql.Row(
        **row.asDict()
        , HDD_type=row.HDD.split(' ')[1]
        )
    )
    .map(lambda row: sql.Row(
        **row.asDict()
        , Volume=row.H * row.D * row.W
        )
    )
    .toDF()
    .select(
        sample_data_df.columns +
        [
              'HDD_size'
            , 'HDD_type'
            , f.round(
                f.col('Volume')
            ).alias('Volume_cuIn')
        ])
)

sample_data_transformed.show()

+---+-----------+----+----------+----+---------+-----+----+----+------+--------+--------+-----------+
| Id|      Model|Year|ScreenSize| RAM|      HDD|    W|   D|   H|Weight|HDD_size|HDD_type|Volume_cuIn|
+---+-----------+----+----------+----+---------+-----+----+----+------+--------+--------+-----------+
|  1|MacBook Pro|2015|       15"|16GB|512GB SSD|13.75|9.48|0.61|  4.02|   512GB|     SSD|       80.0|
|  2|    MacBook|2016|       12"| 8GB|256GB SSD|11.04|7.74|0.52|  2.03|   256GB|     SSD|       44.0|
|  3|MacBook Air|2016|     13.3"| 8GB|128GB SSD| 12.8|8.94|0.68|  2.96|   128GB|     SSD|       78.0|
|  4|       iMac|2017|       27"|64GB|  1TB SSD| 25.6| 8.0|20.3|  20.8|     1TB|     SSD|     4157.0|
+---+-----------+----+----------+----+---------+-----+----+----+------+--------+--------+-----------+



- ¿Para qué utilizamos las librerías importadas?
  - pyspark.sql as sql:
Proporciona clases y funciones para trabajar con Spark SQL, como la clase Row que representa una fila estructurada, y permite crear nuevos objetos Row con campos personalizados.

  - pyspark.sql.functions as f:
Contiene funciones SQL integradas para manipulación de columnas, como round para redondear valores numéricos, col para referenciar columnas en expresiones, entre otras.

- ¿Cambió la estructura del dataset original?
  - Sí, la estructura cambió en el sentido de que:    
    - Se agregaron tres nuevas columnas:

    - HDD_size (string): tamaño extraído de la columna HDD.

    - HDD_type (string): tipo extraído de la columna HDD.

    - Volume_cuIn (numérico): volumen calculado y redondeado.

    - Las columnas originales permanecen intactas, pero ahora el DataFrame tiene más columnas.

- ¿Para qué se utiliza **row.asDict()?
  - row.asDict() convierte la fila row en un diccionario Python con clave-valor, donde las claves son los nombres de columnas y los valores los datos correspondientes.

  - El operador ** desempaqueta ese diccionario para pasar cada par clave-valor como argumento nombrado en la creación de un nuevo objeto Row.

  - Esto permite crear un nuevo Row que contiene todos los campos originales más los nuevos campos que se agregan (como HDD_size, HDD_type, Volume).

  - Es una forma elegante de copiar todos los campos existentes sin listarlos uno a uno y añadir campos adicionales.

## <font color='green'>**Fin ejercicio 1**</font>

## Performance optimizations

A partir de Spark 2.0, el rendimiento de PySpark usando DataFrames estaba a la par con el de Scala o Java. Sin embargo, hubo una excepción: el uso de funciones definidas por el usuario (UDF); Si un usuario define un método Python puro y lo registra como UDF, PySpark tendría que cambiar constantemente los tiempos de ejecución (Python a JVM y viceversa). Esta fue la razón principal de un enorme impacto en el rendimiento en comparación con Scala, que no necesita convertir el objeto JVM en un objeto Python.

Las cosas han cambiado significativamente en Spark 2.3. Primero, Spark comenzó a usar el nuevo proyecto Apache Arrow crea un único espacio de memoria utilizado por todos los entornos, eliminando así la necesidad de copiar y convertir constantemente entre objetos.

<img src="https://drive.google.com/uc?export=view&id=1EjxZ90_9MbK7iZ3YdXyhSEXWQU45DE7S" width=600 height=400/>

En segundo lugar, Arrow almacena objetos columnares en la memoria, lo que aumenta considerablemente el rendimiento. Por lo tanto, para aprovechar aún más eso, partes del código PySpark se han refactorizado y eso nos trajo UDF vectorizadas.

Una función definida por el usuario de pandas(UDF), también conocida como UDF vectorizada, es una función definida por el usuario que usa Apache Arrow para transferir datos y pandas para trabajar con los datos. Las UDF de pandas permiten operaciones vectorizadas que pueden aumentar el rendimiento hasta 100 veces en comparación con las UDF de Python de fila a fila.

In [25]:
import pyspark.sql.functions as f
# from pyspark.sql.functions import pandas_udf, PandasUDFType

# Generamos un DataFrame con un id y un numero aleatorio.
big_df = (
    spark
    .range(0, 1000000)
    .withColumn('val', f.rand())
)

big_df.cache()
big_df.show(3)

+---+------------------+
| id|               val|
+---+------------------+
|  0| 0.272909898721171|
|  1|0.1202477155570687|
|  2|0.9304582076015152|
+---+------------------+
only showing top 3 rows



En este ejemplo, usaremos SciPy para devolver un valor de una función de distribución de probabilidad normal (PDF) para un conjunto de 1,000,000 números aleatorios entre 0 y 1:

El método test_pandas_pdf () simplemente usa el método pandas_pdf(...) para recuperar el PDF de la distribución normal, realiza la operación .count(...) e imprime los resultados usando el método .show(...) .

El método test_pdf() hace lo mismo pero usa el método pdf(...) en su lugar, que es la forma fila por fila de usar las UDF.

El decorador% timeit simplemente ejecuta los métodos test_pandas_pdf () o test_pdf () siete veces, multiplicado por cada ejecución. Aquí hay un resultado de muestra (abreviado como está, como era de esperar, muy repetitivo) para ejecutar el método test_pandas_pdf ():

In [26]:
import pandas as pd
from scipy import stats


@f.pandas_udf('double', f.PandasUDFType.SCALAR)
def pandas_pdf(v):
    return pd.Series(stats.norm.pdf(v))

def test_pandas_pdf():
    return (big_df
            .withColumn('probability', pandas_pdf(big_df.val))
            .agg(f.count(f.col('probability')))
            .show()
        )

%timeit -n 1 test_pandas_pdf()

/content/spark-3.5.5-bin-hadoop3/python/pyspark/sql/pandas/functions.py:407: UserWarning: In Python 3.6+ and Spark 3.0+, it is preferred to specify type hints for pandas UDF instead of specifying pandas UDF type which will be deprecated in the future releases. See SPARK-28264 for more details.
  warnings.warn(


+------------------+
|count(probability)|
+------------------+
|           1000000|
+------------------+

+------------------+
|count(probability)|
+------------------+
|           1000000|
+------------------+

+------------------+
|count(probability)|
+------------------+
|           1000000|
+------------------+

+------------------+
|count(probability)|
+------------------+
|           1000000|
+------------------+

+------------------+
|count(probability)|
+------------------+
|           1000000|
+------------------+

+------------------+
|count(probability)|
+------------------+
|           1000000|
+------------------+

+------------------+
|count(probability)|
+------------------+
|           1000000|
+------------------+

The slowest run took 5.93 times longer than the fastest. This could mean that an intermediate result is being cached.
2.25 s ± 1.72 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [27]:
(
    big_df
    .withColumn('probability', pandas_pdf(big_df.val))
    .show(5)
)

+---+------------------+-------------------+
| id|               val|        probability|
+---+------------------+-------------------+
|  0| 0.272909898721171| 0.3843589337708485|
|  1|0.1202477155570687| 0.3960684259890379|
|  2|0.9304582076015152| 0.2587702255058522|
|  3|0.9290380152313981|   0.25911213687463|
|  4|0.4855491413467007|0.35458133711518086|
+---+------------------+-------------------+
only showing top 5 rows



## Ojo esto demora mucho.

## 🚀 Comparación de rendimiento en PySpark: `pandas_udf` vs `udf`

Cuando se trabaja con Spark, es común definir funciones personalizadas para transformar columnas. Spark permite usar dos tipos principales de funciones definidas por el usuario:

- `@pandas_udf`: funciones vectorizadas, optimizadas con Apache Arrow.
- `@udf`: funciones tradicionales que operan fila por fila.

### ⚡ Ejemplo comparativo

#### 1. Usando `pandas_udf` (rápido)

```python
@f.pandas_udf('double', f.PandasUDFType.SCALAR)
def pandas_pdf(v):
    return pd.Series(stats.norm.pdf(v))

big_df.withColumn('probability', pandas_pdf(big_df.val)).agg(f.count(f.col('probability'))).show()


In [28]:
'''
# La forma anterior, una UDF standar.
@f.udf('double')
def pdf(v):
    return float(stats.norm.pdf(v))

def test_pdf():
    return (big_df
            .withColumn('probability', pdf(big_df.val))
            .agg(f.count(f.col('probability')))
            .show()
        )

%timeit -n 1 test_pdf()
'''

"\n# La forma anterior, una UDF standar.\n@f.udf('double')\ndef pdf(v):\n    return float(stats.norm.pdf(v))\n\ndef test_pdf():\n    return (big_df\n            .withColumn('probability', pdf(big_df.val))\n            .agg(f.count(f.col('probability')))\n            .show()\n        )\n\n%timeit -n 1 test_pdf()\n"

# Using SQL to interact with DataFrames

In [29]:
import pyspark.sql.types as typ

sch = typ.StructType([
      typ.StructField('Id', typ.LongType(), False)
    , typ.StructField('Model', typ.StringType(), True)
    , typ.StructField('Year', typ.IntegerType(), True)
    , typ.StructField('ScreenSize', typ.StringType(), True)
    , typ.StructField('RAM', typ.StringType(), True)
    , typ.StructField('HDD', typ.StringType(), True)
    , typ.StructField('W', typ.DoubleType(), True)
    , typ.StructField('D', typ.DoubleType(), True)
    , typ.StructField('H', typ.DoubleType(), True)
    , typ.StructField('Weight', typ.DoubleType(), True)
])

In [30]:
sample_data_rdd = spark.sparkContext.textFile('/content/drive/MyDrive/DATA SCIENCE UDD 2024/MODULO 8/DataFrames_sample.csv')

header = sample_data_rdd.first()

sample_data_rdd = (
    sample_data_rdd
    .filter(lambda row: row != header)
    .map(lambda row: row.split(','))
    .map(lambda row: (
                int(row[0])
                , row[1]
                , int(row[2])
                , row[3]
                , row[4]
                , row[5]
                , float(row[6])
                , float(row[7])
                , float(row[8])
                , float(row[9])
        )
    )
)

sample_data_schema = spark.createDataFrame(sample_data_rdd, schema=sch)
sample_data_schema.show()

+---+-----------+----+----------+----+---------+-----+----+----+------+
| Id|      Model|Year|ScreenSize| RAM|      HDD|    W|   D|   H|Weight|
+---+-----------+----+----------+----+---------+-----+----+----+------+
|  1|MacBook Pro|2015|    "15\""|16GB|512GB SSD|13.75|9.48|0.61|  4.02|
|  2|    MacBook|2016|    "12\""| 8GB|256GB SSD|11.04|7.74|0.52|  2.03|
|  3|MacBook Air|2016|  "13.3\""| 8GB|128GB SSD| 12.8|8.94|0.68|  2.96|
|  4|       iMac|2017|    "27\""|64GB|  1TB SSD| 25.6| 8.0|20.3|  20.8|
+---+-----------+----+----------+----+---------+-----+----+----+------+



In [31]:
models_df = spark.sparkContext.parallelize([
      ('MacBook Pro', 'Laptop')
    , ('MacBook', 'Laptop')
    , ('MacBook Air', 'Laptop')
    , ('iMac', 'Desktop')
]).toDF(['Model', 'FormFactor'])

models_df.createOrReplaceTempView('models')

In [32]:
sample_data_schema.createOrReplaceTempView('sample_data_view')

In [33]:
spark.sql('''
    SELECT a.*
        , b.FormFactor
    FROM sample_data_view AS a
    LEFT JOIN models AS b
        ON a.Model == b.Model
    ORDER BY Weight DESC
''').show()

+---+-----------+----+----------+----+---------+-----+----+----+------+----------+
| Id|      Model|Year|ScreenSize| RAM|      HDD|    W|   D|   H|Weight|FormFactor|
+---+-----------+----+----------+----+---------+-----+----+----+------+----------+
|  4|       iMac|2017|    "27\""|64GB|  1TB SSD| 25.6| 8.0|20.3|  20.8|   Desktop|
|  1|MacBook Pro|2015|    "15\""|16GB|512GB SSD|13.75|9.48|0.61|  4.02|    Laptop|
|  3|MacBook Air|2016|  "13.3\""| 8GB|128GB SSD| 12.8|8.94|0.68|  2.96|    Laptop|
|  2|    MacBook|2016|    "12\""| 8GB|256GB SSD|11.04|7.74|0.52|  2.03|    Laptop|
+---+-----------+----+----------+----+---------+-----+----+----+------+----------+



In [34]:
spark.sql('''
    SELECT b.FormFactor
        , COUNT(*) AS ComputerCnt
    FROM sample_data_view AS a
    LEFT JOIN models AS b
        ON a.Model == b.Model
    GROUP BY FormFactor
''').show()

+----------+-----------+
|FormFactor|ComputerCnt|
+----------+-----------+
|    Laptop|          3|
|   Desktop|          1|
+----------+-----------+



## <font color='green'>**Ejercicio 2**</font>

En este ejercicio volvemos a trabajar con el archivo people.json.

1. Leemos el archivo como un json
2. Filtramos los registros mayores de 21 años.
3. Agrupamos por edad y contamos la cantidad.

Realicemos algunas consultas SQL

4. Creamos una vista temporal con df.createOrReplaceTempView("people")
5. Realice SELECT * FROM people
6. Realice el select para recuperar las personas mayores de 13 y menores de 25
7. Escriba el dataframe leido desde json en formato parquet
8. Lea el archivo almacenado en parquet y dejelo en un dataframe.


In [35]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [36]:
#Leemos el archivo como un json
sample_data_json_df = (
    spark
    .read
    .json('/content/drive/MyDrive/DATA SCIENCE UDD 2024/MODULO 8/people.json')
)

In [37]:
# Filtramos los registros mayores de 21 años.
sample_data_json_df.filter(sample_data_json_df.age > 21).show()

+---+--------+
|age|    name|
+---+--------+
| 30|    Andy|
| 44|Patricia|
| 89|     Leo|
+---+--------+



In [38]:
#Agrupamos por edad y contamos la cantidad.
sample_data_json_df.groupBy('age').count().show()

+----+-----+
| age|count|
+----+-----+
|  19|    1|
|NULL|    1|
|  89|    1|
|  44|    1|
|  11|    1|
|  30|    1|
+----+-----+



In [39]:
# Creamos una vista temporal con df.createOrReplaceTempView("people")
df = sample_data_json_df
df.createOrReplaceTempView("people")

In [40]:
# Realice SELECT * FROM people
spark.sql("SELECT * FROM people").show()

+----+--------+
| age|    name|
+----+--------+
|NULL| Michael|
|  30|    Andy|
|  19|  Justin|
|  11|     Ana|
|  44|Patricia|
|  89|     Leo|
+----+--------+



In [41]:
# Realice el select para recuperar las personas mayores de 13 y menores de 25
spark.sql("SELECT * FROM people WHERE age > 13 AND age < 25").show()

+---+------+
|age|  name|
+---+------+
| 19|Justin|
+---+------+



In [42]:
# Escriba el dataframe leido desde json en formato parquet
df.write.parquet("people.parquet")

In [43]:
# Lea el archivo almacenado en parquet y dejelo en un dataframe.
df_parquet = spark.read.parquet("people.parquet")
df_parquet.show()

+----+--------+
| age|    name|
+----+--------+
|NULL| Michael|
|  30|    Andy|
|  19|  Justin|
|  11|     Ana|
|  44|Patricia|
|  89|     Leo|
+----+--------+



## <font color='green'>**Fin ejercicio 2**</font>

# Overview of DataFrame transformations

In [44]:
sample_data_schema.cache().show()

+---+-----------+----+----------+----+---------+-----+----+----+------+
| Id|      Model|Year|ScreenSize| RAM|      HDD|    W|   D|   H|Weight|
+---+-----------+----+----------+----+---------+-----+----+----+------+
|  1|MacBook Pro|2015|    "15\""|16GB|512GB SSD|13.75|9.48|0.61|  4.02|
|  2|    MacBook|2016|    "12\""| 8GB|256GB SSD|11.04|7.74|0.52|  2.03|
|  3|MacBook Air|2016|  "13.3\""| 8GB|128GB SSD| 12.8|8.94|0.68|  2.96|
|  4|       iMac|2017|    "27\""|64GB|  1TB SSD| 25.6| 8.0|20.3|  20.8|
+---+-----------+----+----------+----+---------+-----+----+----+------+



In [45]:
sample_data_schema.select('RAM').distinct().show()

+----+
| RAM|
+----+
|16GB|
| 8GB|
|64GB|
+----+



In [46]:
sample_data_schema.dropDuplicates().show()

+---+-----------+----+----------+----+---------+-----+----+----+------+
| Id|      Model|Year|ScreenSize| RAM|      HDD|    W|   D|   H|Weight|
+---+-----------+----+----------+----+---------+-----+----+----+------+
|  2|    MacBook|2016|    "12\""| 8GB|256GB SSD|11.04|7.74|0.52|  2.03|
|  1|MacBook Pro|2015|    "15\""|16GB|512GB SSD|13.75|9.48|0.61|  4.02|
|  3|MacBook Air|2016|  "13.3\""| 8GB|128GB SSD| 12.8|8.94|0.68|  2.96|
|  4|       iMac|2017|    "27\""|64GB|  1TB SSD| 25.6| 8.0|20.3|  20.8|
+---+-----------+----+----------+----+---------+-----+----+----+------+



In [47]:
sample_data_schema.dropna().show()

+---+-----------+----+----------+----+---------+-----+----+----+------+
| Id|      Model|Year|ScreenSize| RAM|      HDD|    W|   D|   H|Weight|
+---+-----------+----+----------+----+---------+-----+----+----+------+
|  1|MacBook Pro|2015|    "15\""|16GB|512GB SSD|13.75|9.48|0.61|  4.02|
|  2|    MacBook|2016|    "12\""| 8GB|256GB SSD|11.04|7.74|0.52|  2.03|
|  3|MacBook Air|2016|  "13.3\""| 8GB|128GB SSD| 12.8|8.94|0.68|  2.96|
|  4|       iMac|2017|    "27\""|64GB|  1TB SSD| 25.6| 8.0|20.3|  20.8|
+---+-----------+----+----------+----+---------+-----+----+----+------+



In [48]:
sample_data_schema.fillna(0).show()

+---+-----------+----+----------+----+---------+-----+----+----+------+
| Id|      Model|Year|ScreenSize| RAM|      HDD|    W|   D|   H|Weight|
+---+-----------+----+----------+----+---------+-----+----+----+------+
|  1|MacBook Pro|2015|    "15\""|16GB|512GB SSD|13.75|9.48|0.61|  4.02|
|  2|    MacBook|2016|    "12\""| 8GB|256GB SSD|11.04|7.74|0.52|  2.03|
|  3|MacBook Air|2016|  "13.3\""| 8GB|128GB SSD| 12.8|8.94|0.68|  2.96|
|  4|       iMac|2017|    "27\""|64GB|  1TB SSD| 25.6| 8.0|20.3|  20.8|
+---+-----------+----+----------+----+---------+-----+----+----+------+



In [49]:
sample_data_schema.filter(sample_data_schema.Year > 2015).show()

+---+-----------+----+----------+----+---------+-----+----+----+------+
| Id|      Model|Year|ScreenSize| RAM|      HDD|    W|   D|   H|Weight|
+---+-----------+----+----------+----+---------+-----+----+----+------+
|  2|    MacBook|2016|    "12\""| 8GB|256GB SSD|11.04|7.74|0.52|  2.03|
|  3|MacBook Air|2016|  "13.3\""| 8GB|128GB SSD| 12.8|8.94|0.68|  2.96|
|  4|       iMac|2017|    "27\""|64GB|  1TB SSD| 25.6| 8.0|20.3|  20.8|
+---+-----------+----+----------+----+---------+-----+----+----+------+



In [50]:
sample_data_schema.freqItems(['RAM']).show()

+-----------------+
|    RAM_freqItems|
+-----------------+
|[16GB, 64GB, 8GB]|
+-----------------+



In [51]:
sample_data_schema.orderBy('W').show()

+---+-----------+----+----------+----+---------+-----+----+----+------+
| Id|      Model|Year|ScreenSize| RAM|      HDD|    W|   D|   H|Weight|
+---+-----------+----+----------+----+---------+-----+----+----+------+
|  2|    MacBook|2016|    "12\""| 8GB|256GB SSD|11.04|7.74|0.52|  2.03|
|  3|MacBook Air|2016|  "13.3\""| 8GB|128GB SSD| 12.8|8.94|0.68|  2.96|
|  1|MacBook Pro|2015|    "15\""|16GB|512GB SSD|13.75|9.48|0.61|  4.02|
|  4|       iMac|2017|    "27\""|64GB|  1TB SSD| 25.6| 8.0|20.3|  20.8|
+---+-----------+----+----------+----+---------+-----+----+----+------+



In [52]:
sample_data_schema.orderBy(f.col('H').desc()).show()

+---+-----------+----+----------+----+---------+-----+----+----+------+
| Id|      Model|Year|ScreenSize| RAM|      HDD|    W|   D|   H|Weight|
+---+-----------+----+----------+----+---------+-----+----+----+------+
|  4|       iMac|2017|    "27\""|64GB|  1TB SSD| 25.6| 8.0|20.3|  20.8|
|  3|MacBook Air|2016|  "13.3\""| 8GB|128GB SSD| 12.8|8.94|0.68|  2.96|
|  1|MacBook Pro|2015|    "15\""|16GB|512GB SSD|13.75|9.48|0.61|  4.02|
|  2|    MacBook|2016|    "12\""| 8GB|256GB SSD|11.04|7.74|0.52|  2.03|
+---+-----------+----+----------+----+---------+-----+----+----+------+



In [53]:
sample_data_schema_rep = (
    sample_data_schema
    .repartition(2, 'Year')
)

sample_data_schema_rep.rdd.getNumPartitions()

2

In [54]:
# Crea sesión de Spark
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").appName("MissingDFExample").getOrCreate()
sc = spark.sparkContext


In [55]:
missing_df = sc.parallelize([
    (None, 36.3, 24.2)
    , (1.6, 32.1, 27.9)
    , (3.2, 38.7, 24.7)
    , (2.8, None, 23.9)
    , (3.9, 34.1, 27.9)
    , (9.2, None, None)
]).toDF(['A', 'B', 'C'])

missing_df.fillna(21.4).show()

+----+----+----+
|   A|   B|   C|
+----+----+----+
|21.4|36.3|24.2|
| 1.6|32.1|27.9|
| 3.2|38.7|24.7|
| 2.8|21.4|23.9|
| 3.9|34.1|27.9|
| 9.2|21.4|21.4|
+----+----+----+



In [56]:
miss_dict = (
    missing_df
    .agg(
        f.mean('A').alias('A')
        , f.mean('B').alias('B')
        , f.mean('C').alias('C')
    )
).toPandas().to_dict('records')[0]

missing_df.fillna(miss_dict).show()

+----+------------------+-----+
|   A|                 B|    C|
+----+------------------+-----+
|4.14|              36.3| 24.2|
| 1.6|              32.1| 27.9|
| 3.2|              38.7| 24.7|
| 2.8|35.300000000000004| 23.9|
| 3.9|              34.1| 27.9|
| 9.2|35.300000000000004|25.72|
+----+------------------+-----+



In [57]:
missing_df.dropna().show()

+---+----+----+
|  A|   B|   C|
+---+----+----+
|1.6|32.1|27.9|
|3.2|38.7|24.7|
|3.9|34.1|27.9|
+---+----+----+



In [58]:
missing_df.dropna(thresh=2).show()

+----+----+----+
|   A|   B|   C|
+----+----+----+
|NULL|36.3|24.2|
| 1.6|32.1|27.9|
| 3.2|38.7|24.7|
| 2.8|NULL|23.9|
| 3.9|34.1|27.9|
+----+----+----+



In [59]:
dupes_df = sc.parallelize([
      (1.6, 32.1, 27.9)
    , (3.2, 38.7, 24.7)
    , (3.9, 34.1, 27.9)
    , (3.2, 38.7, 24.7)
]).toDF(['A', 'B', 'C'])

dupes_df.dropDuplicates().show()

+---+----+----+
|  A|   B|   C|
+---+----+----+
|1.6|32.1|27.9|
|3.2|38.7|24.7|
|3.9|34.1|27.9|
+---+----+----+



In [60]:
sample_data_schema.select('Model', 'ScreenSize').show()

+-----------+----------+
|      Model|ScreenSize|
+-----------+----------+
|MacBook Pro|    "15\""|
|    MacBook|    "12\""|
|MacBook Air|  "13.3\""|
|       iMac|    "27\""|
+-----------+----------+



In [61]:
sample_data_schema.select('W').summary().show()
sample_data_schema.select('W').describe().show()

+-------+------------------+
|summary|                 W|
+-------+------------------+
|  count|                 4|
|   mean|15.797500000000001|
| stddev| 6.630738395281983|
|    min|             11.04|
|    25%|             11.04|
|    50%|              12.8|
|    75%|             13.75|
|    max|              25.6|
+-------+------------------+

+-------+------------------+
|summary|                 W|
+-------+------------------+
|  count|                 4|
|   mean|15.797500000000001|
| stddev| 6.630738395281983|
|    min|             11.04|
|    max|              25.6|
+-------+------------------+



In [62]:
another_macBookPro = sc.parallelize([
      (5, 'MacBook Pro', 2018, '15"', '16GB', '256GB SSD', 13.75, 9.48, 0.61, 4.02)
]).toDF(sample_data_schema.columns)

sample_data_schema.unionAll(another_macBookPro).show()

+---+-----------+----+----------+----+---------+-----+----+----+------+
| Id|      Model|Year|ScreenSize| RAM|      HDD|    W|   D|   H|Weight|
+---+-----------+----+----------+----+---------+-----+----+----+------+
|  1|MacBook Pro|2015|    "15\""|16GB|512GB SSD|13.75|9.48|0.61|  4.02|
|  2|    MacBook|2016|    "12\""| 8GB|256GB SSD|11.04|7.74|0.52|  2.03|
|  3|MacBook Air|2016|  "13.3\""| 8GB|128GB SSD| 12.8|8.94|0.68|  2.96|
|  4|       iMac|2017|    "27\""|64GB|  1TB SSD| 25.6| 8.0|20.3|  20.8|
|  5|MacBook Pro|2018|       15"|16GB|256GB SSD|13.75|9.48|0.61|  4.02|
+---+-----------+----+----------+----+---------+-----+----+----+------+



In [63]:
(
    sample_data_schema
    .withColumn('HDDSplit', f.split(f.col('HDD'), ' '))
    .show()
)

+---+-----------+----+----------+----+---------+-----+----+----+------+------------+
| Id|      Model|Year|ScreenSize| RAM|      HDD|    W|   D|   H|Weight|    HDDSplit|
+---+-----------+----+----------+----+---------+-----+----+----+------+------------+
|  1|MacBook Pro|2015|    "15\""|16GB|512GB SSD|13.75|9.48|0.61|  4.02|[512GB, SSD]|
|  2|    MacBook|2016|    "12\""| 8GB|256GB SSD|11.04|7.74|0.52|  2.03|[256GB, SSD]|
|  3|MacBook Air|2016|  "13.3\""| 8GB|128GB SSD| 12.8|8.94|0.68|  2.96|[128GB, SSD]|
|  4|       iMac|2017|    "27\""|64GB|  1TB SSD| 25.6| 8.0|20.3|  20.8|  [1TB, SSD]|
+---+-----------+----+----------+----+---------+-----+----+----+------+------------+



In [64]:
(
    sample_data_schema
    .groupBy('RAM')
    .count()
    .show()
)

+----+-----+
| RAM|count|
+----+-----+
|16GB|    1|
| 8GB|    2|
|64GB|    1|
+----+-----+



In [65]:
(
    sample_data_schema
    .select(
        f.col('*')
        , f.split(f.col('HDD'), ' ').alias('HDD_Array')
    ).show()
)

+---+-----------+----+----------+----+---------+-----+----+----+------+------------+
| Id|      Model|Year|ScreenSize| RAM|      HDD|    W|   D|   H|Weight|   HDD_Array|
+---+-----------+----+----------+----+---------+-----+----+----+------+------------+
|  1|MacBook Pro|2015|    "15\""|16GB|512GB SSD|13.75|9.48|0.61|  4.02|[512GB, SSD]|
|  2|    MacBook|2016|    "12\""| 8GB|256GB SSD|11.04|7.74|0.52|  2.03|[256GB, SSD]|
|  3|MacBook Air|2016|  "13.3\""| 8GB|128GB SSD| 12.8|8.94|0.68|  2.96|[128GB, SSD]|
|  4|       iMac|2017|    "27\""|64GB|  1TB SSD| 25.6| 8.0|20.3|  20.8|  [1TB, SSD]|
+---+-----------+----+----------+----+---------+-----+----+----+------+------------+



# Overview of DataFrame actions

In [66]:
sample_data_schema.groupBy('Year').count().collect()

[Row(Year=2015, count=1), Row(Year=2016, count=2), Row(Year=2017, count=1)]

In [67]:
sample_data_schema.select('W').describe().show()

+-------+------------------+
|summary|                 W|
+-------+------------------+
|  count|                 4|
|   mean|15.797500000000001|
| stddev| 6.630738395281983|
|    min|             11.04|
|    max|              25.6|
+-------+------------------+



In [68]:
sample_data_schema.take(2)

[Row(Id=1, Model='MacBook Pro', Year=2015, ScreenSize='"15\\""', RAM='16GB', HDD='512GB SSD', W=13.75, D=9.48, H=0.61, Weight=4.02),
 Row(Id=2, Model='MacBook', Year=2016, ScreenSize='"12\\""', RAM='8GB', HDD='256GB SSD', W=11.04, D=7.74, H=0.52, Weight=2.03)]

In [69]:
sample_data_schema.toPandas()

,Id,Model,Year,ScreenSize,RAM,HDD,W,D,H,Weight
0,1,MacBook Pro,2015,"""15\""""",16GB,512GB SSD,13.75,9.48,0.61,4.02
1,2,MacBook,2016,"""12\""""",8GB,256GB SSD,11.04,7.74,0.52,2.03
2,3,MacBook Air,2016,"""13.3\""""",8GB,128GB SSD,12.80,8.94,0.68,2.96
3,4,iMac,2017,"""27\""""",64GB,1TB SSD,25.60,8.00,20.30,20.80


In [70]:
sample_data_schema.write.mode('overwrite').csv('sample_data_schema.csv')

## <font color='green'>**Ejercicio 3**</font>
### 🧠 Análisis “fitness” del catálogo de Macs

**Dataset:** `sample_data_schema` (o `sample_data_df`)

---

### ⚙️ Potencia-peso

- Crea una nueva columna `Weight_lbs` que represente el peso en libras.
- Crea una columna `PowerRatio` definida como:

  $$
  \text{PowerRatio} = \frac{\text{RAM (GB)}}{\text{Weight (kg)}}
  $$

📌 *Pista:* usa `regexp_replace` para extraer el número de la columna `RAM`.

---

### 📅 Clasificación por décadas

- Añade una nueva columna `Decade` a partir de la columna `Year`, con valores como `"2010's"`, `"2020's"`, etc.

---

### 📊 Comparativa

- Agrupa los datos por `Decade`.
- Calcula la **media** y la **desviación estándar** de `PowerRatio` para cada grupo.
- Ordena los resultados de mayor a menor según la media de `PowerRatio` para identificar en qué década los Macs tuvieron la mejor relación **potencia/masa**.

---

### 📈 Visualización rápida *(opcional)*

- Convierte el resultado a un DataFrame de **Pandas**.
- Realiza un **gráfico de barras** con `matplotlib` para ilustrar la evolución de la potencia relativa por década.


In [71]:
# Aqui su codigo
# utilizar dataset sample_data_schema
sample_data_schema.show()

+---+-----------+----+----------+----+---------+-----+----+----+------+
| Id|      Model|Year|ScreenSize| RAM|      HDD|    W|   D|   H|Weight|
+---+-----------+----+----------+----+---------+-----+----+----+------+
|  1|MacBook Pro|2015|    "15\""|16GB|512GB SSD|13.75|9.48|0.61|  4.02|
|  2|    MacBook|2016|    "12\""| 8GB|256GB SSD|11.04|7.74|0.52|  2.03|
|  3|MacBook Air|2016|  "13.3\""| 8GB|128GB SSD| 12.8|8.94|0.68|  2.96|
|  4|       iMac|2017|    "27\""|64GB|  1TB SSD| 25.6| 8.0|20.3|  20.8|
+---+-----------+----+----------+----+---------+-----+----+----+------+



In [72]:
# Crea una nueva columna Weight_lbs que represente el peso en libras.
sample_data_schema = sample_data_schema.withColumn('Weight_lbs', f.round(f.col('Weight') * 2.20462, 2).cast('int'))
sample_data_schema.show()

+---+-----------+----+----------+----+---------+-----+----+----+------+----------+
| Id|      Model|Year|ScreenSize| RAM|      HDD|    W|   D|   H|Weight|Weight_lbs|
+---+-----------+----+----------+----+---------+-----+----+----+------+----------+
|  1|MacBook Pro|2015|    "15\""|16GB|512GB SSD|13.75|9.48|0.61|  4.02|         8|
|  2|    MacBook|2016|    "12\""| 8GB|256GB SSD|11.04|7.74|0.52|  2.03|         4|
|  3|MacBook Air|2016|  "13.3\""| 8GB|128GB SSD| 12.8|8.94|0.68|  2.96|         6|
|  4|       iMac|2017|    "27\""|64GB|  1TB SSD| 25.6| 8.0|20.3|  20.8|        45|
+---+-----------+----+----------+----+---------+-----+----+----+------+----------+



In [73]:
# obtener el valor numérico de columna RAM
sample_data_schema = sample_data_schema.withColumn('RAM_num', f.regexp_replace(f.col('RAM'), '[^0-9]', '').cast('int'))
# Crea una columna PowerRatio definida como: PowerRatio=RAM(GB) / Weight (kg).

sample_data_schema = sample_data_schema.withColumn('PowerRatio', f.round(f.col('RAM_num') / f.col('Weight'),2).cast('float'))
sample_data_schema.show()

+---+-----------+----+----------+----+---------+-----+----+----+------+----------+-------+----------+
| Id|      Model|Year|ScreenSize| RAM|      HDD|    W|   D|   H|Weight|Weight_lbs|RAM_num|PowerRatio|
+---+-----------+----+----------+----+---------+-----+----+----+------+----------+-------+----------+
|  1|MacBook Pro|2015|    "15\""|16GB|512GB SSD|13.75|9.48|0.61|  4.02|         8|     16|      3.98|
|  2|    MacBook|2016|    "12\""| 8GB|256GB SSD|11.04|7.74|0.52|  2.03|         4|      8|      3.94|
|  3|MacBook Air|2016|  "13.3\""| 8GB|128GB SSD| 12.8|8.94|0.68|  2.96|         6|      8|       2.7|
|  4|       iMac|2017|    "27\""|64GB|  1TB SSD| 25.6| 8.0|20.3|  20.8|        45|     64|      3.08|
+---+-----------+----+----------+----+---------+-----+----+----+------+----------+-------+----------+



In [74]:
# Añade una nueva columna Decade a partir de la columna Year, con valores como "2010's", "2020's", etc
sample_data_schema = sample_data_schema.withColumn('Decade', f.when(f.col('Year') < 2020, '2010s').cast('char(10)'))
sample_data_schema.show()

+---+-----------+----+----------+----+---------+-----+----+----+------+----------+-------+----------+------+
| Id|      Model|Year|ScreenSize| RAM|      HDD|    W|   D|   H|Weight|Weight_lbs|RAM_num|PowerRatio|Decade|
+---+-----------+----+----------+----+---------+-----+----+----+------+----------+-------+----------+------+
|  1|MacBook Pro|2015|    "15\""|16GB|512GB SSD|13.75|9.48|0.61|  4.02|         8|     16|      3.98| 2010s|
|  2|    MacBook|2016|    "12\""| 8GB|256GB SSD|11.04|7.74|0.52|  2.03|         4|      8|      3.94| 2010s|
|  3|MacBook Air|2016|  "13.3\""| 8GB|128GB SSD| 12.8|8.94|0.68|  2.96|         6|      8|       2.7| 2010s|
|  4|       iMac|2017|    "27\""|64GB|  1TB SSD| 25.6| 8.0|20.3|  20.8|        45|     64|      3.08| 2010s|
+---+-----------+----+----------+----+---------+-----+----+----+------+----------+-------+----------+------+



In [75]:
# Agrupa los datos por Decade.
sample_data_schema.groupBy('Decade').count().show()

+------+-----+
|Decade|count|
+------+-----+
| 2010s|    4|
+------+-----+



In [76]:
# Calcula la media y la desviación estándar de PowerRatio para cada grupo.
sample_data_schema.groupBy('Decade').agg(f.mean('PowerRatio').alias('mean_PowerRatio'), f.stddev('PowerRatio').alias('std_PowerRatio')).show()

+------+-----------------+------------------+
|Decade|  mean_PowerRatio|    std_PowerRatio|
+------+-----------------+------------------+
| 2010s|3.425000011920929|0.6371551521176215|
+------+-----------------+------------------+



In [77]:
# Ordena los resultados de mayor a menor según la media de PowerRatio para identificar en qué década los Macs tuvieron la mejor relación potencia/masa.
sample_data_schema.groupBy('Decade').agg(f.mean('PowerRatio').alias('mean_PowerRatio'), f.stddev('PowerRatio').alias('std_PowerRatio')).orderBy('mean_PowerRatio', ascending=False).show()

+------+-----------------+------------------+
|Decade|  mean_PowerRatio|    std_PowerRatio|
+------+-----------------+------------------+
| 2010s|3.425000011920929|0.6371551521176215|
+------+-----------------+------------------+



## <font color='green'>**Fin ejercicio 3**</font>

# <font color='purple' style='bold' size=5>**EXPERIMENTO** </font>
Experimentar más calculos del dataframe

In [89]:
from pyspark.sql import functions as f

sample_data_schema.groupBy('Decade').agg(
    f.mean('PowerRatio').alias('mean_PowerRatio'),
    f.stddev('PowerRatio').alias('std_PowerRatio'),
    f.min('PowerRatio').alias('min_PowerRatio'),
    f.max('PowerRatio').alias('max_PowerRatio'),
    f.count('PowerRatio').alias('count_PowerRatio')
).show()


+------+-----------------+------------------+--------------+--------------+----------------+
|Decade|  mean_PowerRatio|    std_PowerRatio|min_PowerRatio|max_PowerRatio|count_PowerRatio|
+------+-----------------+------------------+--------------+--------------+----------------+
| 2010s|3.425000011920929|0.6371551521176215|           2.7|          3.98|               4|
+------+-----------------+------------------+--------------+--------------+----------------+



In [90]:
# Calcular mediana aproximada para cada grupo es un poco más complejo,
# aquí un ejemplo para toda la columna (sin agrupar):

median = sample_data_schema.approxQuantile("PowerRatio", [0.5], 0.01)
print(f"Mediana aproximada global de PowerRatio: {median[0]}")

Mediana aproximada global de PowerRatio: 3.0799999237060547


In [91]:
sample_data_schema.groupBy('Decade').agg(
    f.sum('PowerRatio').alias('sum_PowerRatio')
).show()

+------+------------------+
|Decade|    sum_PowerRatio|
+------+------------------+
| 2010s|13.700000047683716|
+------+------------------+



In [92]:
sample_data_schema = sample_data_schema.filter(col('PowerRatio') > 0)

sample_data_schema.groupBy('Decade').agg(
    f.exp(f.sum(f.log('PowerRatio'))).alias('product_PowerRatio')
).show()

+------+------------------+
|Decade|product_PowerRatio|
+------+------------------+
| 2010s|130.40486079161292|
+------+------------------+



### <font color='purple' style='bold' >**FIN EXPERIMENTO** </font>

---
## <font color='green'>**Ejercicio 4**</font>

### 📡  — Simulación de “stream” y ventana temporal

**Dataset:** `big_df` (1 millón de números aleatorios)

Este ejercicio simula un flujo de datos tipo **IoT** para practicar funciones de ventana y agregaciones temporales en PySpark.

---

### 🔍 Marcar eventos “altos”

- Crea una nueva columna booleana `is_high` que valga `1` si `val > 0.9`, y `0` en caso contrario.

---

### 🕒 Timestamp artificial

- Añade una columna `event_time` para simular un **timestamp escalonado** artificialmente:
  - Usa `expr("current_timestamp()")` como base de tiempo actual.
  - Aplica `expr("cast(id/10000 as int) seconds")` para espaciar los eventos por bloques de tiempo.

---

### ⏳ Ventana deslizante

- Agrupa usando una **ventana de tiempo deslizante** para contar cuántos eventos “altos” caen en cada intervalo:
  
  ```python
  groupBy(window("event_time", "10 seconds", "5 seconds"))

  para contar cuántos eventos altos caen en cada ventana de 10 s, desplazada cada 5 s.


### 🔝 Top N ventanas

Ordena por el conteo descendente y muestra las 5 ventanas con mayor número de “picos”.

---




In [94]:
#Aqui su codigo
big_df.show()

+---+-------------------+-------+----------+-------------------+
| id|                val|is_high|event_time|      event_time_ts|
+---+-------------------+-------+----------+-------------------+
|  0|  0.272909898721171|      0|         0|1970-01-01 00:00:00|
|  1| 0.1202477155570687|      0|         0|1970-01-01 00:00:00|
|  2| 0.9304582076015152|      1|         0|1970-01-01 00:00:00|
|  3| 0.9290380152313981|      1|         0|1970-01-01 00:00:00|
|  4| 0.4855491413467007|      0|         0|1970-01-01 00:00:00|
|  5|0.37782291078106023|      0|         0|1970-01-01 00:00:00|
|  6| 0.2210981937320099|      0|         0|1970-01-01 00:00:00|
|  7| 0.5256298872336228|      0|         0|1970-01-01 00:00:00|
|  8| 0.7476121323287866|      0|         0|1970-01-01 00:00:00|
|  9| 0.2317201359248402|      0|         0|1970-01-01 00:00:00|
| 10|  0.350637205353583|      0|         0|1970-01-01 00:00:00|
| 11|0.03858810012628655|      0|         0|1970-01-01 00:00:00|
| 12| 0.5696491022150654|

In [95]:
# Crea una nueva columna booleana is_high que valga 1 si val > 0.9, y 0 en caso contrario.
big_df = big_df.withColumn('is_high', f.when(f.col('val') > 0.9, 1).otherwise(0))
big_df.show()

+---+-------------------+-------+----------+-------------------+
| id|                val|is_high|event_time|      event_time_ts|
+---+-------------------+-------+----------+-------------------+
|  0|  0.272909898721171|      0|         0|1970-01-01 00:00:00|
|  1| 0.1202477155570687|      0|         0|1970-01-01 00:00:00|
|  2| 0.9304582076015152|      1|         0|1970-01-01 00:00:00|
|  3| 0.9290380152313981|      1|         0|1970-01-01 00:00:00|
|  4| 0.4855491413467007|      0|         0|1970-01-01 00:00:00|
|  5|0.37782291078106023|      0|         0|1970-01-01 00:00:00|
|  6| 0.2210981937320099|      0|         0|1970-01-01 00:00:00|
|  7| 0.5256298872336228|      0|         0|1970-01-01 00:00:00|
|  8| 0.7476121323287866|      0|         0|1970-01-01 00:00:00|
|  9| 0.2317201359248402|      0|         0|1970-01-01 00:00:00|
| 10|  0.350637205353583|      0|         0|1970-01-01 00:00:00|
| 11|0.03858810012628655|      0|         0|1970-01-01 00:00:00|
| 12| 0.5696491022150654|

In [96]:
from pyspark.sql.functions import col, from_unixtime, expr
#Aplica expr("cast(id/10000 as int) seconds") para espaciar los eventos por bloques de tiempo.
big_df = big_df.withColumn('event_time', expr("cast(id/10000 as int)"))

big_df = big_df.withColumn("event_time_ts", from_unixtime(col("event_time")).cast("timestamp"))
big_df.show()

+---+-------------------+-------+----------+-------------------+
| id|                val|is_high|event_time|      event_time_ts|
+---+-------------------+-------+----------+-------------------+
|  0|  0.272909898721171|      0|         0|1970-01-01 00:00:00|
|  1| 0.1202477155570687|      0|         0|1970-01-01 00:00:00|
|  2| 0.9304582076015152|      1|         0|1970-01-01 00:00:00|
|  3| 0.9290380152313981|      1|         0|1970-01-01 00:00:00|
|  4| 0.4855491413467007|      0|         0|1970-01-01 00:00:00|
|  5|0.37782291078106023|      0|         0|1970-01-01 00:00:00|
|  6| 0.2210981937320099|      0|         0|1970-01-01 00:00:00|
|  7| 0.5256298872336228|      0|         0|1970-01-01 00:00:00|
|  8| 0.7476121323287866|      0|         0|1970-01-01 00:00:00|
|  9| 0.2317201359248402|      0|         0|1970-01-01 00:00:00|
| 10|  0.350637205353583|      0|         0|1970-01-01 00:00:00|
| 11|0.03858810012628655|      0|         0|1970-01-01 00:00:00|
| 12| 0.5696491022150654|

In [97]:
from pyspark.sql.functions import col, from_unixtime, expr, window, sum as _sum
import pyspark.sql.functions as f


big_df = big_df.withColumn('event_time', expr("cast(id/10000 as int)"))

big_df = big_df.withColumn("event_time_ts", from_unixtime(col("event_time")).cast("timestamp"))


big_df.groupBy(
    window("event_time_ts", "10 seconds", "5 seconds")
).agg(
    _sum("is_high").alias("count_high")
).show()


+--------------------+----------+
|              window|count_high|
+--------------------+----------+
|{1970-01-01 00:00...|      9944|
|{1970-01-01 00:00...|      9996|
|{1970-01-01 00:00...|     10036|
|{1970-01-01 00:00...|     10163|
|{1970-01-01 00:00...|      9927|
|{1970-01-01 00:00...|      9983|
|{1970-01-01 00:00...|     10069|
|{1969-12-31 23:59...|      5103|
|{1970-01-01 00:00...|      9909|
|{1970-01-01 00:00...|     10008|
|{1970-01-01 00:00...|      9921|
|{1970-01-01 00:01...|     10047|
|{1970-01-01 00:01...|     10076|
|{1970-01-01 00:01...|     10120|
|{1970-01-01 00:01...|     10065|
|{1970-01-01 00:00...|      9865|
|{1970-01-01 00:01...|      5019|
|{1970-01-01 00:01...|     10047|
|{1970-01-01 00:00...|      9915|
|{1970-01-01 00:01...|      9984|
+--------------------+----------+
only showing top 20 rows



### <font color='green'>**Fin ejercicio 4**</font>

# <font color='purple' style='bold' size=5>**MATERIAL ADICIONAL** </font>
Para profundizar con una guía sobre DataFrames de Pyspark:
- [Tutorial de Pyspark: Primeros pasos con Pyspark
](https://www.datacamp.com/es/tutorial/pyspark-tutorial-getting-started-with-pyspark): Tutorial introductorio sobre PySpark incluyendo la manipulación y uso de DataFrames.

### <font color='purple' style='bold' >**FIN MATERIAL ADICIONAL** </font>